# 유튜브 콘텐츠 크롤링
- 참고 : [깃허브](https://github.com/park-gb/youtube-content-scaper)

In [1]:
# 필요 모듈 다운로드
! pip install bs4==0.0.1
! pip install selenium==4.1.2
! pip install webdriver-manager==3.5.3
! pip install pandas==1.4.1
! pip install numpy==1.22.2

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1264 sha256=0578cd989d41d9f978c827b3c62f8743cdcf1b51e9af1f9b0dcf3497acbcf1b3
  Stored in directory: c:\users\hop09\appdata\local\pip\cache\wheels\73\2b\cb\099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4
                                              0.0/963.5 kB ? eta -:--:--
     ------------------------------------  962.6/963.5 kB 30.7 MB/s eta 0:00:01
     ------------------------------------- 963.5/963.5 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: selenium
    Found existing installation: selenium 4.12.0
    Uninstalling selenium-4.12.0:
      Successfully uninstalled selenium-4.12.0
  Attempting uninstall: webdriver-manager
    Found existing installation: webdriver-manager 3.8.6
    Uninstalling webdriver-manager-3.8.6:
      Successfully uninstalled webdriver-manager-3.8.6
           

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
librosa 0.10.0.post2 requires numpy!=1.22.0,!=1.22.1,!=1.22.2,>=1.20.3, but you have numpy 1.22.2 which is incompatible.
tensorboard 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow-intel 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 2.10.0 which is incompatible.
tensorflow-intel 2.12.0 requires tensorboard<2.13,>=2.12, but you have tensorboard 2.10.0 which is incompatible.
tensorflow-intel 2.12.0 requires tensorflow-estimator<2.13,>=2.12.0, but you have tensorflow-esti

# 패키지 Import

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [10]:
# 최신버전 업데이트 매번 크롬드라이버 다운 불필요
! pip install selenium -U 


In [4]:
import time
import random
import pandas as pd

# 무한 스크롤 함수
## 1) 기능
- 콘텐츠 로딩을 충분히 기다리며 스크롤이 불가할 때까지 스크롤을 무한 반복하는 함수

## 2) 역할
- 유튜브 웹 페이지는 스크롤을 해야 새로운 콘텐츠 정보를 제공하기 때문에, 모든 검색 결과를 확인하기 위해서는 무한 스크롤 기능 필요

In [5]:
def scroll():
    try:        
        # 페이지 내 스크롤 높이 받아오기
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            # 임의의 페이지 로딩 시간 설정
            # PC환경에 따라 로딩시간 최적화를 통해 scraping 시간 단축 가능
            pause_time = random.uniform(1, 2)
            # 페이지 최하단까지 스크롤
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤(i.e., 페이지를 위로 올렸다가 내리는 제스쳐)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight-50)")
            time.sleep(pause_time)
            # 페이지 내 스크롤 높이 새롭게 받아오기
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우)
            if new_page_height == last_page_height:
                print("스크롤 완료")
                break
                
            # 스크롤 완료하지 않은 경우, 최하단까지 스크롤
            else:
                last_page_height = new_page_height
            
    except Exception as e:
        print("에러 발생: ", e)

# 데이터 Scrap

In [6]:
# 검색 키워드 설정: 키워드 내 띄어쓰기는 URL에서 '+'로 표시되기 때문에 이에 맞게 변환
SEARCH_KEYWORD = '투자종목 추천'.replace(' ', '+')

In [12]:
driver = webdriver.Chrome()
# 스크래핑 할 URL 세팅
URL = "https://www.youtube.com/results?search_query=" + SEARCH_KEYWORD
# 크롬 드라이버를 통해 지정한 URL의 웹 페이지 오픈
driver.get(URL)
# 웹 페이지 로딩 대기
time.sleep(3)
# 무한 스크롤 함수 실행
scroll()

스크롤 완료


In [13]:
# 페이지 소스 추출
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

# 데이터 추출

In [19]:
# 콘텐츠 모든 정보
content_total = soup_source.find_all(class_ = 'yt-simple-endpoint style-scope ytd-video-renderer')
# 콘텐츠 제목만 추출
content_total_title = list(map(lambda data: data.get_text().replace("\n", ""), content_total))
# 콘텐츠 링크만 추출
content_total_link = list(map(lambda data: "https://youtube.com" + data["href"], content_total))

#--------조회수 & 업로드 날짜 추출(Updated at 2022-10-11)--------#
content_record_src = soup_source.find_all(class_ = 'inline-metadata-item style-scope ytd-video-meta-block')
content_view_cnt = [content_record_src[i].get_text().replace('조회수 ', '') for i in range(0, len(content_record_src), 10)]
content_upload_date = [content_record_src[i].get_text() for i in range(1, len(content_record_src), 10)]
#---------------------------------------------------------#

# 데이터프레임 저장

In [21]:
import os
path = os.getcwd()
print(path)

c:\Users\hop09\Desktop\SEMINA\미래에셋-유튜브


In [16]:
# 모든 리스트가 같은 길이를 가지도록 확인
min_length = min(len(content_total_title), len(content_total_link), len(content_view_cnt), len(content_upload_date))
content_total_dict = {
    'title': content_total_title[:min_length],
    'link': content_total_link[:min_length],
    'view': content_view_cnt[:min_length],
    'upload_date': content_upload_date[:min_length]
}

df = pd.DataFrame(content_total_dict)
df.to_csv(path+"/content_total.csv", encoding='utf-8-sig')

# 데이터 확인

In [17]:
df

,title,link,view,upload_date
0,"지금부터 '이 주식' 꼭 사야 한다, 놓치면 반드시 후회합니다 (전업투자자 김정수)",https://youtube.com/watch?v=TmTSczyw6lg&pp=ygU...,7.7만회,1개월 전
1,2차전지 다음으로 3년간 큰돈 벌 기회가 왔어요 '이 주식' 사 모으세요 (장기 투...,https://youtube.com/watch?v=qhAABUhN_ms&pp=ygU...,14만회,조회수 53만회
2,국내 ETF주식 세 종목만 모으셔도 부자가 됩니다 !,https://youtube.com/watch?v=FCK-Wl2cjpo&pp=ygU...,33만회,조회수 326만회
3,국내 배당주식 세 종목만 모으셔도 부자가 됩니다!,https://youtube.com/watch?v=1AIl51rqe4w&pp=ygU...,39만회,1년 전
4,"2023년 배당주 추천 종목 완벽 정리! (투자 방법, 배당금 많은 주식 순위, 고...",https://youtube.com/watch?v=Mq1XCWg3hvk&pp=ygU...,5.5천회,5개월 전
...,...,...,...,...
139,"주식초보 주린이를 위한 1 Tier 추천 도서 완벽 정리, 한권으로 주식투자 공부 ...",https://youtube.com/watch?v=-7PNSj9m1XA&pp=ygU...,4.2만회,5개월 전
140,국내 배당주식 대신 유튜브 한 종목 추천으로 큰일날뻔..(feat.부자and거지),https://youtube.com/watch?v=qugJE7bsksg&pp=ygU...,246회,3개월 전
141,단기투자 vs 장기투자,https://youtube.com/shorts/5_AdtXEWgyU,없음,1일 전
142,장투하기 좋은 1000원대 주식 유망종목 4,https://youtube.com/watch?v=UnGjGK0o178&pp=ygU...,15만회,9시간 전
